In [1]:
from keras.applications.xception import Xception
from keras.layers import Input, Flatten, Dense, GlobalAveragePooling2D, Dropout
from keras.models import Model
from keras.optimizers import RMSprop, SGD
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import time

Using TensorFlow backend.


In [2]:
nb_epoch = 20
batch_size = 32
lr = 1e-4
img_height=299
img_width=299
nb_train_samples = 3019
nb_validation_samples = 758
train_data_dir = '/Users/jasonmancuso/iPython Directory/Kaggle/Nature Conservancy/train_split'
val_data_dir = '/Users/jasonmancuso/iPython Directory/Kaggle/Nature Conservancy/val_split'
top_model_weights_path = "top_model.h5"
FishNames = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']

In [3]:
# Build Xception network
print "Loading Xception weights..."
Xception_notop = Xception(include_top=False, weights='imagenet', input_tensor=None)
print "Weights loaded."

Loading Xception weights...
Weights loaded.


In [4]:
input = Input(shape=(10,10,2048))
x = GlobalAveragePooling2D()(input)
x = Dense(256, activation='relu')(x)
x = Dropout(.5)(x)
predictions = Dense(8,activation='softmax')(x)
top_model = Model(input,predictions)
top_model.load_weights(top_model_weights_path)

In [5]:
output = top_model(Xception_notop.output)
Xception_full = Model(Xception_notop.input,output)

In [6]:
for layer in Xception_full.layers[-7:]:
    layer.trainable=False
Xception_full.compile(loss='categorical_crossentropy',
             optimizer=SGD(lr=lr,momentum=0.9,nesterov=True),
             metrics=['accuracy','categorical_crossentropy'])
#Xception_full.fit(train_features,train_labels,
#             nb_epoch=nb_epoch, batch_size=batch_size,
#             validation_data=(val_features,val_labels))
#time.sleep(0.1)

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    rotation_range=10.,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    classes=FishNames)

validation_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    classes=FishNames)

Xception_full.load_weights("Xception_full_epoch30_valcc0.2492.h5")

checkpoint_file = "./Xception_full.{epoch:02d}-{val_loss:.2f}.hdf5"
checkpointer = ModelCheckpoint(checkpoint_file, monitor='val_loss', verbose = 1)

# fine-tune the model
Xception_full.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    nb_epoch=nb_epoch,
    validation_data=validation_generator,
    callbacks=[checkpointer]
    nb_val_samples=nb_validation_samples)

Found 3019 images belonging to 8 classes.
Found 758 images belonging to 8 classes.
Epoch 1/20
3019/3019 [==============================] - 5092s - loss: 0.2623 - acc: 0.9089 - categorical_crossentropy: 0.2623 - val_loss: 0.2574 - val_acc: 0.9235 - val_categorical_crossentropy: 0.2574
Epoch 2/20
3019/3019 [==============================] - 5082s - loss: 0.2490 - acc: 0.9175 - categorical_crossentropy: 0.2490 - val_loss: 0.2827 - val_acc: 0.9129 - val_categorical_crossentropy: 0.2827
Epoch 3/20
3019/3019 [==============================] - 5055s - loss: 0.2433 - acc: 0.9149 - categorical_crossentropy: 0.2433 - val_loss: 0.2557 - val_acc: 0.9182 - val_categorical_crossentropy: 0.2557
Epoch 4/20
3019/3019 [==============================] - 5049s - loss: 0.2315 - acc: 0.9248 - categorical_crossentropy: 0.2315 - val_loss: 0.2347 - val_acc: 0.9261 - val_categorical_crossentropy: 0.2347
Epoch 5/20
3019/3019 [==============================] - 5076s - loss: 0.2099 - acc: 0.9308 - categorical_cros

In [8]:
import os
test_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True)

root_path = '/Users/jasonmancuso/iPython Directory/Kaggle/Nature Conservancy/'
test_data_dir = os.path.join(root_path, 'test_stg1')
nbr_augmentation=5
nbr_test_samples=1000

In [9]:
for idx in range(nbr_augmentation):
    print('{}th augmentation for testing ...'.format(idx))
    random_seed = np.random.random_integers(0, 100000)

    test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        shuffle = False, # Important !!!
        seed = random_seed,
        classes = None,
        class_mode = None)

    test_image_list = test_generator.filenames
    #print('image_list: {}'.format(test_image_list[:10]))
    print('Begin to predict for testing data ...')
    if idx == 0:
        predictions = Xception_full.predict_generator(test_generator, nbr_test_samples)
    else:
        predictions += Xception_full.predict_generator(test_generator, nbr_test_samples)

predictions /= nbr_augmentation

print('Begin to write submission file ..')
f_submit = open(os.path.join(root_path, 'submit.csv'), 'w')
f_submit.write('image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT\n')
for i, image_name in enumerate(test_image_list):
    pred = ['%.6f' % p for p in predictions[i, :]]
    if i % 100 == 0:
        print('{} / {}'.format(i, nbr_test_samples))
    f_submit.write('%s,%s\n' % (os.path.basename(image_name), ','.join(pred)))

f_submit.close()

print('Submission file successfully generated!')

0th augmentation for testing ...
Found 1000 images belonging to 1 classes.
Begin to predict for testing data ...


/Users/jasonmancuso/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: DeprecationWarning: This function is deprecated. Please call randint(0, 100000 + 1) instead
  app.launch_new_instance()


1th augmentation for testing ...
Found 1000 images belonging to 1 classes.
Begin to predict for testing data ...
2th augmentation for testing ...
Found 1000 images belonging to 1 classes.
Begin to predict for testing data ...
3th augmentation for testing ...
Found 1000 images belonging to 1 classes.
Begin to predict for testing data ...
4th augmentation for testing ...
Found 1000 images belonging to 1 classes.
Begin to predict for testing data ...
Begin to write submission file ..
0 / 1000
100 / 1000
200 / 1000
300 / 1000
400 / 1000
500 / 1000
600 / 1000
700 / 1000
800 / 1000
900 / 1000
Submission file successfully generated!


In [10]:
Xception_full.save("Xception_full.epoch50.hdf5")